In [8]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

In [2]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        # return score, tag_seq
        return tag_seq

In [50]:
# data
import numpy as np
import pandas as pd

PARTICIPANT_ID = 24

df_csv=pd.read_csv('/Users/apple/Desktop/deep_eye/dataset/input_data.csv',usecols=['participant_ID','Q_ID','text','fixation1'])
df_csv=df_csv[(df_csv['participant_ID']==PARTICIPANT_ID)] # take 1st participant data
df_csv=df_csv.drop(['participant_ID'],axis=1)

In [28]:
total_q_list = []
for q in range(47):
    df_q = df_csv[df_csv['Q_ID'] == q+1]
    text_list = list(df_q['text'])
    label_list = list(df_q['fixation1'])
    q_list = [text_list, label_list]
    total_q_list.append(q_list)

print(
    '\n',total_q_list[46], # first data with text and label
    '\n',total_q_list[46][0], # first data's text
    '\n',total_q_list[46][1], # first data's label
)



 [['將', '硫酸鈉', '溶液', '加熱', '時', '燒杯', '內', '硫酸鈉', '的', '溶解度', '會', '有', '什麼', '變化', '變大', '變小', '不會', '改變'], [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0]] 
 ['將', '硫酸鈉', '溶液', '加熱', '時', '燒杯', '內', '硫酸鈉', '的', '溶解度', '會', '有', '什麼', '變化', '變大', '變小', '不會', '改變'] 
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0]


In [29]:
word_to_ix = {}
for sentence, tags in total_q_list:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

In [30]:
word_to_ix_df = pd.DataFrame.from_dict([word_to_ix]).T
word_to_ix_df = word_to_ix_df.reset_index().iloc[:,0]
word_to_ix_df.to_csv("/Users/apple/Desktop/deep_eye/dataset/word_to_index.csv")
word_to_ix_df

0        兩
1        個
2       量筒
3        各
4        裝
      ... 
367     加熱
368     燒杯
369    溶解度
370     變大
371     變小
Name: index, Length: 372, dtype: object

---

In [56]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

textindex_list = []
for q in range(47):
    df_q = df_csv[df_csv['Q_ID'] == q+1]
    precheck_sent = prepare_sequence(list(df_q['text']), word_to_ix)
    textindex_list.append(precheck_sent)
textindex_list

[tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  5,  6,  9, 10, 11, 12, 13,  6, 14,
         15,  6, 16, 15,  6, 17, 15,  6]),
 tensor([18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 21, 30, 31, 24, 25, 26,
         30, 32, 21, 30, 31, 24, 25, 26, 30, 33]),
 tensor([34, 35, 36, 35, 37, 38, 39, 40, 21, 41, 31, 42, 43, 44, 45, 46, 39, 37,
         47, 48]),
 tensor([49, 50, 34, 51, 52, 53, 54, 55, 56, 34, 57, 58, 59, 60, 59, 50, 59, 61,
         59]),
 tensor([62, 63, 64, 65, 66, 67, 68, 34, 68, 69, 44, 70, 71, 72, 73,  7, 27, 74,
         75, 76, 77, 78,  7, 79,  7, 44, 45, 80,  3, 81]),
 tensor([82,  0, 83, 20, 84, 73, 85, 86, 11, 87, 88,  1, 89, 90, 91, 11, 92, 88,
         83, 85, 27, 93, 94, 31, 73, 93, 94, 95, 73, 93, 94, 96, 97, 98, 94]),
 tensor([ 99,  73, 100, 101,  74,  13,   1, 102,  88,   1,   0,   1, 103,   1]),
 tensor([104, 105, 106, 107, 108,  66,   8, 109, 107, 110, 111, 104, 106, 109,
         112,   1, 107,  73, 113, 114, 115, 116, 104, 117, 106, 117, 109, 104,
         

In [84]:
wordindex_df = pd.DataFrame([x.tolist() for x in textindex_list])
wordindex_df.index = list(range(1,48))
wordindex_df.to_csv("/Users/apple/Desktop/deep_eye/dataset/word_to_index_df.csv")
wordindex_df


,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,39,40,41,42,43
1,0,1,2,3,4,5,6,7,8,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,18,19,20,21,22,23,24,25,26,27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,34,35,36,35,37,38,39,40,21,41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,49,50,34,51,52,53,54,55,56,34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,62,63,64,65,66,67,68,34,68,69,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,82,0,83,20,84,73,85,86,11,87,...,94.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,99,73,100,101,74,13,1,102,88,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,104,105,106,107,108,66,8,109,107,110,...,106.0,119.0,109.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,34,120,54,121,7,73,122,123,124,27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,126,129,130,131,1,132,133,101,134,135,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
list(wordindex_df[(wordindex_df == 0).any(axis=1)].index)

[1, 6, 7, 12, 14, 18, 28]

In [83]:
indexinlist = []
for i in range(word_to_ix_df.shape[0]): # 372
    indexinlist.append(list(wordindex_df[(wordindex_df == i).any(axis=1)].index))
indexinlist_df = pd.DataFrame(indexinlist)
indexinlist_df

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,1,6.0,7.0,12.0,14.0,18.0,28.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,6.0,7.0,8.0,10.0,12.0,15.0,20.0,24.0,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,5.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,21.0,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
368,47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
369,47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
370,47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
indexinlist_df.to_csv("/Users/apple/Desktop/deep_eye/dataset/index_in_sentence_df.csv")

---

In [6]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 4 #5
HIDDEN_DIM = 4 #4


training_data = total_q_list#[:37]
#testing_data = total_q_list[37:]

# word_to_ix # unique word and its representing number
word_to_ix = {}
for sentence, tags in training_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

tag_to_ix = {0: 0, 1: 1, START_TAG: 2, STOP_TAG: 3}

model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

# Check predictions before training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    precheck_tags = torch.tensor([tag_to_ix[t] for t in training_data[0][1]], dtype=torch.long)
    print('real label：'+ str(precheck_tags.tolist()))
    print('untrained label prediction：'+ str(model(precheck_sent)))

print('=============start BiLSTM+CRF model training=============')
# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in range(200):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()
    if epoch%50 == 0:
        print(f'the{epoch}th epoch Loss:{loss[0]}')


output_path = 'ner_trained_model.cpt'
torch.save(model, output_path)
print('=============trained model saved=============\n\n')



print('=============load trained model and predict=============')
model_path = 'ner_trained_model.cpt'
trained_ner_model = torch.load(model_path)
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    print('trained label prediction：' + str(model(precheck_sent)))

# time spent : 3min

real label：[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0]
untrained label prediction：[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]
=============start BiLSTM+CRF model training=============
the0th epoch Loss:11.823930740356445
the50th epoch Loss:8.182518005371094
the100th epoch Loss:7.828432083129883
the150th epoch Loss:3.9106616973876953
=============trained model saved=============


=============load trained model and predict=============
trained label prediction：[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0]


In [10]:
# total_wrong = []
# acccuracy_score = []
# for q in range(47):
#     precheck_sent = prepare_sequence(training_data[q][0], word_to_ix)
#     predict = model(precheck_sent)
#     answer = training_data[q][1]
#     accuracy = sum([1 for i,j in zip(predict,answer) if i==j])/len(answer)
#     print(str(q+1) + " accuracy:", round(accuracy,2))
#     acccuracy_score.append(accuracy)
#     wrong_list = [i for i,j in zip(predict,answer) if i!=j] # wrong answer
#     total_wrong.append(wrong_list)

In [7]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# training predict score
total_result_list = []
total_output_list = []
for q in range(len(training_data)):
    precheck_sent = prepare_sequence(training_data[q][0], word_to_ix)
    predict = model(precheck_sent)
    answer = training_data[q][1]
    output_list = [training_data[q][0], predict, answer]
    total_output_list.append(output_list)
    # print(training_data[q][0])
    # print(predict)
    # print(answer)
    accuracy = accuracy_score(answer, predict)
    precision = precision_score(answer, predict)
    recall = recall_score(answer, predict)
    f1score = f1_score(answer, predict)
    result_list = [accuracy, precision, recall, f1score]
    result_list = [round(elem, 2) for elem in result_list]
    total_result_list.append(result_list)

In [18]:
total_qresult_df = pd.DataFrame()
for q in range(47):
    q_result = pd.DataFrame(total_output_list[q])
    total_qresult_df = pd.concat([total_qresult_df, q_result],axis=0)
total_qresult_df.to_csv("/Users/apple/Desktop/deep_eye/dataset/P_ID_24_qresult.csv")
total_qresult_df

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,39,40,41,42,43
0,兩,個,量筒,各,裝,50,cc,水,與,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,1,1,1,1,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,1,1,1,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,沙罐,置入,不同,濃度,鹽水,中,所,受,浮力,會,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,1,1,1,1,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,0,0,1,1,1,1,0,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,1,1,1,1,0,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,將,硫酸鈉,溶液,加熱,時,燒杯,內,硫酸鈉,的,溶解度,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,1,0,0,0,0,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
total_result_df = pd.DataFrame(total_result_list)
total_result_df.columns = ['accuracy', 'precision', 'recall', 'f1score']
total_result_df

,accuracy,precision,recall,f1score
0,0.92,0.90,1.00,0.95
1,0.82,0.80,0.73,0.76
2,0.95,1.00,0.93,0.96
3,0.95,0.88,1.00,0.93
4,0.97,1.00,0.95,0.97
5,0.94,0.93,0.93,0.93
6,0.79,0.80,0.89,0.84
7,0.86,1.00,0.58,0.74
8,1.00,1.00,1.00,1.00
9,0.97,0.95,1.00,0.97


In [11]:
total_result_df.mean(axis=0)

accuracy     0.919787
precision    0.902766
recall       0.905106
f1score      0.894894
dtype: float64

---

In [34]:
word_to_ix = {}
for sentence, tags in testing_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

In [35]:
total_result_list = []
for q in range(len(testing_data)):
    precheck_sent = prepare_sequence(testing_data[q][0], word_to_ix)
    predict = model(precheck_sent)
    answer = testing_data[q][1]
    accuracy = accuracy_score(answer, predict)
    precision = precision_score(answer, predict)
    recall = recall_score(answer, predict)
    f1score = f1_score(answer, predict)
    result_list = [accuracy, precision, recall, f1score]
    result_list = [round(elem, 2) for elem in result_list]
    total_result_list.append(result_list)

In [36]:
total_result_df = pd.DataFrame(total_result_list)
total_result_df.columns = ['accuracy', 'precision', 'recall', 'f1score']
total_result_df

,accuracy,precision,recall,f1score
0,0.37,0.22,0.29,0.25
1,0.44,0.60,0.41,0.49
2,0.38,0.45,0.26,0.33
3,0.52,0.33,0.40,0.36
4,0.58,0.54,0.64,0.58
5,0.78,0.60,1.00,0.75
6,0.50,0.40,0.44,0.42
7,0.63,0.45,0.56,0.50
8,0.52,0.64,0.50,0.56
9,0.67,0.88,0.58,0.70


In [37]:
total_result_df.mean(axis=0)

accuracy     0.539
precision    0.511
recall       0.508
f1score      0.494
dtype: float64

In [41]:
# total_result_df.iloc[-1] = total_result_df.mean(axis=0)
# total_result_df = total_result_df.rename(index={9: 'mean'})
# total_result_df

,accuracy,precision,recall,f1score
0,0.37000,0.22000,0.29000,0.25000
1,0.44000,0.60000,0.41000,0.49000
2,0.38000,0.45000,0.26000,0.33000
3,0.52000,0.33000,0.40000,0.36000
4,0.58000,0.54000,0.64000,0.58000
5,0.78000,0.60000,1.00000,0.75000
6,0.50000,0.40000,0.44000,0.42000
7,0.63000,0.45000,0.56000,0.50000
8,0.52000,0.64000,0.50000,0.56000
mean,0.52459,0.47041,0.50008,0.47134


In [38]:
total_result_list = []
for q in range(len(testing_data)):
    precheck_sent = prepare_sequence(testing_data[q][0], word_to_ix)
    predict = model(precheck_sent)
    answer = testing_data[q][1]
    print(testing_data[q][0])
    print(predict)
    print(answer)
    print("===============================")
    # accuracy = accuracy_score(answer, predict)
    # precision = precision_score(answer, predict)
    # recall = recall_score(answer, predict)
    # f1score = f1_score(answer, predict)
    # result_list = [accuracy, precision, recall, f1score]
    # result_list = [round(elem, 2) for elem in result_list]
    # total_result_list.append(result_list)

['未成熟', '的', '香蕉', '和', '成熟', '香蕉', '加', '碘液', '後', '哪', '個', '會', '比較', '黑', '呢', '未成熟', '成熟', '一樣', '黑']
[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0]
['將', '鋅', '與', '電池', '正極', '連接', '銅', '與', '電池', '負極', '連接', '放', '在', '硫酸鋅', '溶液', '中', '電極', '會', '產生', '什麼', '變化', '銅棒', '上', '有', '鋅', '鋅棒', '上', '有', '銅', '二', '者', '都', '產生', '氣體']
[0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1]
[0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
['量筒', '裝滿', '水', '後', '上方', '放上', '大小', '恰', '可', '套入', '的', '保麗龍球', '垂直', '倒立', '後', '會', '有', '何', '現象', '球', '往', '下', '掉', '球', '向', '上', '跑', '球', '停', '在', '量筒', '口']
[0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1]
[0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0]
['A', 'B', 'C',

In [18]:
# # draw confusion matrix
# import matplotlib.pyplot as plt
# conf_matrix = confusion_matrix(y_true=answer, y_pred=predict)
# fig, ax = plt.subplots(figsize=(5, 5))
# ax.matshow(conf_matrix, cmap=plt.cm.Oranges, alpha=0.3)
# for i in range(conf_matrix.shape[0]):
#     for j in range(conf_matrix.shape[1]):
#         ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='xx-large')

# plt.xlabel('Predictions', fontsize=18)
# plt.ylabel('Actuals', fontsize=18)
# plt.title('Confusion Matrix', fontsize=18)
# plt.show()